# Entropy Mapping

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
import geopandas as gpd
from shapely import Point
from scipy.ndimage.filters import gaussian_filter
import contextily as ctx
from shapely.geometry import Polygon
import os
%matplotlib widget

/Users/cpoliziani/opt/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.3-CAPI-1.17.3) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_99002/3154988767.py:8: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


In [10]:
# Some variables to declare and modify based on user preferences

# Time/Bin length
binLength = 1800 #By default, BEAMCORE outputs time in seconds, so for hour-length bins use 3600

mainDirectory  = '/Users/cpoliziani/Downloads/entropyCalc/'#In my case I have the codes in a directory and the images are in a sub-directory

minParkTime = 600

# zoningFilepath = 'BlockIDS/geo_export_2642a579-6c47-479f-b94c-0d5f27ccb433.shp'
zoningFilepath = 'Grids/grid1000.shp'



In [16]:
#CP I've put all the functions here at the beginning
def create_grid(xmin, xmax, ymin, ymax, grid_size, crs="EPSG:3857", output_file="grid.shp"):
    # Generate the coordinates for the grid
    x_coords = np.arange(xmin, xmax, grid_size)
    y_coords = np.arange(ymin, ymax, grid_size)

    # Create polygons for each grid cell
    polygons = []
    for x in x_coords:
        for y in y_coords:
            # Define the polygon corners
            poly = Polygon([(x, y), (x + grid_size, y), 
                            (x + grid_size, y + grid_size), (x, y + grid_size)])
            polygons.append(poly)

    # Create a GeoDataFrame with the grid polygons
    grid = gpd.GeoDataFrame({'geometry': polygons}, crs=crs)
    
    grid['geoid'] = range(len(grid))
    
    # Save the grid to a shapefile
    grid.to_file(output_file)
    
def parkingTimecol(df):
    """
    Calculate the parking time for each vehicle trip in the DataFrame.

    Parameters:
    df: A DataFrame containing daily vehicle information, including
                       'vehicle', 'departureTime', and 'arrivalTime' columns.

    Returns:
    A new DataFrame with an additional column 'Parking Time' 
                  representing the total parking time for each vehicle.
    """
    vehicleList = df['vehicle'].unique()
    column = []
    for item in vehicleList:
        subdf = df.loc[df['vehicle'] == item].copy() 
        subdf = subdf.sort_values(by = 'departureTime', ignore_index = True)
        firstDepart = subdf['departureTime'].iloc[0]
        subdf['departureTime'] = subdf['departureTime'].shift(-1) 
        subdf.loc[subdf.index[-1], 'departureTime'] = firstDepart + 24 * 3600  
        subdf.loc[:, 'Parking Time'] = subdf['departureTime'] - subdf['arrivalTime'] 
        column.append(subdf)
    return pd.concat(column)  

def travelLengthCol(df):
    """
    Calculate the daily cumulative travel length for each vehicle trip in the DataFrame.

    Parameters:
    df: A DataFrame containing vehicle information, including
                       'vehicle' and 'length' columns.

    Returns:
    A new DataFrame with an additional column 'Cumulative Travel Length' 
                  representing the total distance traveled by each vehicle.
    """
    vehicleList = df['vehicle'].unique()
    distanceTraveled = []
    for item in vehicleList:
        subdf = df.loc[df['vehicle'] == item].copy() 
        subdf = subdf.sort_values(by = 'departureTime', ignore_index = True)
        subdf.loc[:, 'Cumulative Travel Length'] = subdf['length'].cumsum()  
        distanceTraveled.append(subdf)
    return pd.concat(distanceTraveled) 

def timeCorrecter(df):
    """
    Transform Parking time column into a list of parking times per time bin
    Time bins start from 0 and advance by binLength
    
    """
    listoflists = []
    for index, row in df.iterrows():
        listalike = []
        parkResult = divmod(row['Parking Time'], binLength)
        arriveResult = divmod(row['arrivalTime'], binLength)
        i = 0
        while i < parkResult[0]:
            listalike.append(binLength)
            i += 1
        listalike.append(parkResult[1])
        listalike[0] = listalike[0] - arriveResult[1]
        listalike[-1] = listalike[-1] + arriveResult[1]
        if listalike[-1] > binLength:
            listalike[-1] = listalike[-1] - binLength
            listalike.insert(-1, binLength)
        listoflists.append(listalike)
    return listoflists



#CP I'd separate the entropy calculation from the entropy plot functions: have a unique entropy calculation file, and multiple plot functions

def entropyPlot5(result_gdf):
    #CP groupby geoid - might want to groupy by a new variable that you specify at the beginning as an input
    #e.g. groupVariable = 'geoid' or 'link'

    fig, ax = plt.subplots(figsize=(8, 8))
    plt.suptitle('Heatmap of Entropy by Location', y = 0.95)
    plt.title(time)

    img = result_gdf.plot(column = 'Entropy',
                                cmap = 'Reds',
                                ax = ax,
                                legend = True,
                                vmin = 0,
                                vmax = 0.05
                                )
#SF     xlim=(-13650000, -13600000)
#SF     ylim=(4530000, 4570000)    
    xlim=(-13755000, -13485000)
    ylim=(4420000, 4705000)
    
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    plt.savefig(mainDirectory + 'images/heatmap' + str(time) +'.png', dpi = 100)
    




## Grabbing the BEAM CORE Data

In [5]:
########### Activate only when creating new grids

# # Create a grid shapefile
# # Inputs for the grid creation (you can easily change these)
# xmin = -13755000 # X min boundary
# xmax = -13485000# X max boundary
# ymin = 4420000    # Y min boundary
# ymax = 4705000  # Y max boundary
# grid_size = 500# Grid size in meters (1km)
# name = 'grid500'

# # Call the function to create the grid and save as shapefile
# create_grid(xmin, xmax, ymin, ymax, grid_size, crs='EPSG:3857', output_file= mainDirectory+'Grids/'+name+'.shp')

# Read and filter data
events = pd.read_csv('https://storage.googleapis.com/beam-core-outputs/sfbay-tr_capacity_1_5-20230608/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz', 
                     usecols = [
#                          'time',
                                'length',
                                'arrivalTime',
                                'departureTime',
                                'primaryFuelType',
                                'vehicle',
                                'mode',
                                'type', 
#                                 'link',
#                                 'links',
#                                 'linkTravelTime', 
#                                 'startX',
#                                 'startY',
                                'endX',
                                'endY'],
#                     nrows = 10000000 # to have a smaller DF
                    )

#Slicing and merging dataframes
print('Total Events: ', len(events))
PTs = events.loc[events['type'] == 'PathTraversal']
print('Total PathTraversal: ', len(PTs))
PTs = PTs.drop(columns = 'type')

PTs = PTs[PTs['arrivalTime']<104400]
print('Total PathTraversal after filtering Arrival < 5am of Successive Day: ', len(PTs))


print('Simulated PathTraversal Modes: ',PTs['mode'].value_counts())
#More slicing, this time specifically isolating the electric cars in the events files
PTs = PTs.loc[events['mode'].isin(['car',
                                         'car_hov2',
                                         'car_hov3',
#                                          'ride_hail',
#                                          'ride_hail_pooled',
                                        ])]
print('Total Length Car PathTraversal: ', len(PTs))
PTs = PTs.loc[PTs['primaryFuelType'] == 'Electricity']
print('Total Length Electric Car PathTraversal: ', len(PTs))
PTs = PTs.drop(columns = ['mode','primaryFuelType'])

#Sorting the rows by vehicle and then by departure time, making the accumulation sums easier to manage
PTs = PTs.sort_values(by = ['vehicle', 'departureTime'], ignore_index = True)

print('Total Travel Length in km',PTs['length'].sum()/1000)


PTs.to_csv(mainDirectory + 'elecCarTrace.csv')

/var/folders/h9/g8d27p9s5js3dx9z7b5y8k5c0000gr/T/ipykernel_99002/791406502.py:3: DtypeWarning: Columns (24,25,59) have mixed types. Specify dtype option on import or set low_memory=False.
  events = pd.read_csv('https://storage.googleapis.com/beam-core-outputs/sfbay-tr_capacity_1_5-20230608/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz',


Total Events:  36807426
Total PathTraversal:  8968315
Total PathTraversal after filtering Arrival < 5m of Successive Day:  8955881
Simulated PathTraversal Modes:  walk         4153525
car          2693381
bus          1285026
car_hov2      407358
car_hov3      287333
tram           54929
bike           40360
cable_car      17836
subway         14358
rail            1702
ferry             73
Name: mode, dtype: int64
Total Length Car PathTraversal:  3388072
Total Length Electric Car PathTraversal:  11116
Total Travel Length in km 71445.74804699999


## Sorting and Calculating

In [17]:
PTs2 = PTs.copy()
# Calculate Parking Time
PTs2 = parkingTimecol(PTs2)
print('Total Parking Time in Hours',PTs2['Parking Time'].sum()/3600)

# Calculate Cumulative Length
PTs2 = travelLengthCol(PTs2)
PTs2 = PTs2.drop(columns = ['length'])

# Calculate Last Link
# elecCarTrace['last_link'] = elecCarTrace['links'].str.split(',').apply(lambda x: x[-1])
# elecCarTrace = elecCarTrace.drop(columns=['links'])

#Filter small parking time and zero lengths
PTs2 = PTs2[PTs2['Parking Time']>minParkTime]
PTs2 = PTs2[PTs2['Cumulative Travel Length']>0]

# Make sure it's still in order
PTs2 = PTs2.sort_values(by=['vehicle', 'departureTime'], ignore_index=True)

PTs2.reset_index(drop = True, inplace = True)

PTs2.to_csv(mainDirectory + 'elecCarTrace2.csv')

In [18]:
elecCarTrace = PTs2.copy()

binLabel = np.divmod(elecCarTrace['arrivalTime'], binLength)
elecCarTrace['Bin Label'] = binLabel[0]


# Transfor the parking time column in lists of parking times per bin
elecCarTrace['Parking Time'] = timeCorrecter(elecCarTrace)
elecCarTrace = elecCarTrace.sort_values(by=['vehicle', 'departureTime'], ignore_index=True)

def assign_sequence(lst):
    return [(i, val) for i, val in enumerate(lst)]  # (sequence_number, value)

# Apply the function to 'Parking Time' to associate a sequence number
elecCarTrace['Parking Time'] = elecCarTrace['Parking Time'].apply(assign_sequence)

# Separate the sequence number and the 'Parking Time' into two columns
elecCarTrace = elecCarTrace.explode('Parking Time')
elecCarTrace['sequence_number'], elecCarTrace['Parking Time'] = zip(*elecCarTrace['Parking Time'])
elecCarTrace['Bin Label'] += elecCarTrace['sequence_number']
# Optionally reset the index if needed and sort if necessary
elecCarTrace.reset_index(drop=True, inplace=True)


#Transfor Bin column into actual time in seconds
elecCarTrace['Bin Label'] = elecCarTrace['Bin Label'] * binLength

#Calculate Charging Opportunity
elecCarTrace['Charging Opportunity'] = elecCarTrace['Parking Time'] * elecCarTrace['Cumulative Travel Length']
elecCarTrace['Charging Opportunity'] = elecCarTrace['Charging Opportunity'].astype(float)

#Add up time bins from the second day to the first day
elecCarTrace.loc[elecCarTrace['Bin Label'] >= 24*3600 , 'Bin Label'] -= 24 *3600
print('Total Chargin Opportunities Entries:', len(elecCarTrace))
elecCarTrace = elecCarTrace[elecCarTrace['Bin Label']<=24*3600]
print('Total Chargin Opportunities Entries after filtering people departing for the first time after midnight:', len(elecCarTrace))


#CP moved this here
#Add block group info and normalize by area
#CP try differnt geometries
elecCarTrace = gpd.GeoDataFrame(elecCarTrace, geometry=gpd.points_from_xy(elecCarTrace['endX'], elecCarTrace['endY']))
elecCarTrace = elecCarTrace.set_crs(epsg=4326)
elecCarTrace = elecCarTrace.to_crs(epsg=3857)
block_groups_gdf = gpd.read_file(zoningFilepath)[['geoid','geometry']]
if block_groups_gdf.crs != elecCarTrace.crs: block_groups_gdf = block_groups_gdf.to_crs(elecCarTrace.crs)
elecCarTrace = gpd.sjoin(elecCarTrace, block_groups_gdf[['geoid','geometry']], how='left', op='within')
print('Total Chargin Opportunities Entries with Valid Block Group:', len(elecCarTrace))
elecCarTrace = elecCarTrace.drop(columns=['geometry'])
elecCarTrace = elecCarTrace.merge(block_groups_gdf[['geoid','geometry']], how ='left', on = 'geoid')
elecCarTrace = gpd.GeoDataFrame(elecCarTrace, geometry='geometry')


elecCarTrace.to_csv(mainDirectory + 'elecCarTrace3.csv')

## The Different Functions for Entropy Plots

In [19]:
total_entropy_list = []
total_opportunity_list = []
time_labels = []

for time in elecCarTrace['Bin Label'].sort_values().unique():
    
    print('Time', time)
    time_labels.append(time) 
    
    subdf = elecCarTrace.loc[elecCarTrace['Bin Label'] == time]
    
    result_gdf = subdf.groupby(['geoid']).agg(
        {'Charging Opportunity': 'sum', 
        'geometry': 'first', 
        }    ).reset_index()
    
    result_gdf = gpd.GeoDataFrame(result_gdf, geometry='geometry')
    
    totalChargingOpportunity = subdf['Charging Opportunity'].sum()
    print('Total Charging Opportunity', totalChargingOpportunity)
    total_opportunity_list.append(totalChargingOpportunity)
    
    #Calculate Entropy of charging opportunity density
    result_gdf['Density Charging Opportunity'] = result_gdf['Charging Opportunity'].div(result_gdf.geometry.area)
    totalJunk = result_gdf['Density Charging Opportunity'].sum()
    result_gdf['Probability'] = result_gdf['Density Charging Opportunity'] / totalJunk
    result_gdf['Entropy'] = -result_gdf['Probability'] * np.log(result_gdf['Probability'])
    
    totalEntropy = result_gdf['Entropy'].sum()
    print('Total Entropy', totalEntropy)
    total_entropy_list.append(totalEntropy)

    entropyPlot5(result_gdf)
    

#Plot Entropy
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot total charging opportunity on the first axis
ax1.set_xlabel('Time')
ax1.set_ylabel('Total Charging Opportunity', color='tab:blue')
ax1.plot(time_labels, total_opportunity_list, label='Total Charging Opportunity', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create a second y-axis for the total entropy
ax2 = ax1.twinx()  # Instantiate a second y-axis that shares the same x-axis
ax2.set_ylabel('Total Entropy', color='tab:red')
ax2.plot(time_labels, total_entropy_list, label='Total Entropy', color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Add a title and display the plot
plt.title('Temporal Evolution of Total Charging Opportunity and Total Entropy')
fig.tight_layout()  # To prevent overlap of labels
plt.savefig(mainDirectory + 'timeEvolution.png', dpi = 100)
plt.show()

In [20]:
image_dir = 'images'
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')]
image_files = sorted(image_files, key=os.path.getmtime, reverse=False)

fig, ax = plt.subplots()
# Function to update the plot for each frame
def animate(i):
    ax.clear()
    image = plt.imread(image_files[i])
    ax.set_axis_off()
    ax.imshow(image, extent = None)

# Create the animation
ani = FuncAnimation(fig, animate, frames=len(image_files), interval=200)

# Save the animation
ani.save(mainDirectory + '/animation.gif', writer='pillow')